# first hard problem: split IDs by location

In [122]:
import pandas as pd

In [3]:
# import raw minute-by-minute dockless bike status
raw = pd.read_csv('../raw_data/dockless_bike_data_2022-01-31.csv')

In [13]:
raw.columns.values

array(['Unnamed: 0', 'bike_id', 'is_reserved', 'is_disabled', 'type',
       'name', 'lon', 'lat', 'rental_uris.android', 'rental_uris.ios',
       'timestamp'], dtype=object)

In [60]:
# note columns to exclude from final scraper
drop_cols = ['Unnamed: 0', 
             'type',
             'rental_uris.android', 
             'rental_uris.ios']

# define columns to keep here
cols = ['bike_id', 
        'lon', 
        'lat', 
        'timestamp']

df = raw[cols]

In [193]:
# select subset of records with no duplicate timestamps or partial scrapes
df_slice = df.iloc[:19359].copy()

# for each bike record, identify previous most recent timestamp
df_slice['prev_time'] = df_slice.sort_values(by='timestamp').groupby('bike_id')['timestamp'].shift()

# convert string times to timedelta
# NOTE: will not be needed with proper timestamps; TODO: fix scraper
df_slice['timestamp'] = pd.to_timedelta(df_slice['timestamp'])
df_slice['prev_time'] = pd.to_timedelta(df_slice['prev_time'])

# calculate time since bike last seen
df_slice['time_diff'] = (df_slice['timestamp'] - df_slice['prev_time']).dt.total_seconds()

# create counter incrementing each time bike reappears after missing at least one scrape
df_slice['bike_instance'] = (
    df_slice.sort_values(by='timestamp')
    .groupby('bike_id')['time_diff']
    .apply(lambda x: (x > 62).cumsum())
)

## TODO: figure out what to do about bikes that had time gaps but didn't move

In [ ]:
# df_slice.to_csv('slice.csv')

In [194]:
df_slice

,bike_id,lon,lat,timestamp,prev_time,time_diff,id2,bike_instance
0,95f9e94200eb485380f93055e2d18137,-77.014537,38.878164,0 days 00:52:39,NaT,NaN,0,0
1,b1d307df75032bef781bd6374cf93f61,-76.952504,38.906637,0 days 00:52:39,NaT,NaN,0,0
2,663be416fe27b395b2a19479a682d4b9,-77.109047,38.878765,0 days 00:52:39,NaT,NaN,0,0
3,512c0183912eaab4081ca1a91a6e765e,-77.040034,38.907323,0 days 00:52:39,NaT,NaN,0,0
4,c2cbe6f86fef02cd18a1b3166eec00ae,-77.017614,38.877089,0 days 00:52:39,NaT,NaN,0,0
...,...,...,...,...,...,...,...,...
19354,3ba312a5f10f58ad72a18bdbb04e08bd,-76.946861,38.959550,0 days 02:31:48,0 days 02:30:47,61.0,0,0
19355,13127be40a20ca8cb8059585661d5221,-77.011109,38.873086,0 days 02:31:48,0 days 02:30:47,61.0,0,0
19356,29c7878767f5c9a3c0b6a4b9a0a99fa5,-76.966213,38.942586,0 days 02:31:48,0 days 02:30:47,61.0,0,0
19357,f64922f225d96e6a97b95967c46f5261,-77.121461,38.877651,0 days 02:31:48,0 days 02:30:47,61.0,0,0


In [189]:
df_slice.query('bike_id == "96cd65315ee3fe78cfcccdda68b38f11"')

,bike_id,lon,lat,timestamp,prev_time,time_diff,id2
10849,96cd65315ee3fe78cfcccdda68b38f11,-77.041567,38.905694,0 days 01:48:15,NaT,NaN,0
11061,96cd65315ee3fe78cfcccdda68b38f11,-77.041567,38.905694,0 days 01:49:16,0 days 01:48:15,61.0,0
11183,96cd65315ee3fe78cfcccdda68b38f11,-77.041574,38.905705,0 days 01:50:17,0 days 01:49:16,61.0,0
11554,96cd65315ee3fe78cfcccdda68b38f11,-77.041553,38.905730,0 days 01:52:18,0 days 01:50:17,121.0,1
11903,96cd65315ee3fe78cfcccdda68b38f11,-77.041553,38.905730,0 days 01:53:19,0 days 01:52:18,61.0,1
12083,96cd65315ee3fe78cfcccdda68b38f11,-77.041605,38.905764,0 days 01:54:19,0 days 01:53:19,60.0,1


In [187]:
df_slice.query('bike_id == "88f1400c881f7c064bc08ea6500e180c"')

,bike_id,lon,lat,timestamp,prev_time,time_diff,id2
14394,88f1400c881f7c064bc08ea6500e180c,-76.999049,38.923245,0 days 02:06:29,NaT,NaN,0
14476,88f1400c881f7c064bc08ea6500e180c,-76.999049,38.923245,0 days 02:07:29,0 days 02:06:29,60.0,0
17936,88f1400c881f7c064bc08ea6500e180c,-76.999030,38.923270,0 days 02:24:42,0 days 02:07:29,1033.0,1


In [188]:
df_slice.query('bike_id == "74d7ba8e97c6f64a9c69c92119b20f76"')

,bike_id,lon,lat,timestamp,prev_time,time_diff,id2
1656,74d7ba8e97c6f64a9c69c92119b20f76,-77.108860,38.878691,0 days 01:00:44,NaT,NaN,0
1804,74d7ba8e97c6f64a9c69c92119b20f76,-77.108829,38.878616,0 days 01:01:45,0 days 01:00:44,61.0,0
2086,74d7ba8e97c6f64a9c69c92119b20f76,-77.108859,38.878698,0 days 01:02:45,0 days 01:01:45,60.0,0
2210,74d7ba8e97c6f64a9c69c92119b20f76,-77.108859,38.878657,0 days 01:03:46,0 days 01:02:45,61.0,0
2411,74d7ba8e97c6f64a9c69c92119b20f76,-77.108859,38.878657,0 days 01:04:47,0 days 01:03:46,61.0,0
2632,74d7ba8e97c6f64a9c69c92119b20f76,-77.108879,38.878697,0 days 01:05:48,0 days 01:04:47,61.0,0
5941,74d7ba8e97c6f64a9c69c92119b20f76,-77.108850,38.878888,0 days 01:22:59,0 days 01:05:48,1031.0,1
6206,74d7ba8e97c6f64a9c69c92119b20f76,-77.108760,38.878799,0 days 01:24:00,0 days 01:22:59,61.0,1
6347,74d7ba8e97c6f64a9c69c92119b20f76,-77.108760,38.878799,0 days 01:25:00,0 days 01:24:00,60.0,1
6493,74d7ba8e97c6f64a9c69c92119b20f76,-77.108790,38.878825,0 days 01:26:01,0 days 01:25:00,61.0,1


In [191]:
df_slice.query('bike_id == "533e83e32965e21c8eefc4925ad84b02"')

,bike_id,lon,lat,timestamp,prev_time,time_diff,id2
13410,533e83e32965e21c8eefc4925ad84b02,-77.059292,38.862833,0 days 02:01:25,NaT,NaN,0
13650,533e83e32965e21c8eefc4925ad84b02,-77.059146,38.862764,0 days 02:02:26,0 days 02:01:25,61.0,0
18336,533e83e32965e21c8eefc4925ad84b02,-77.059110,38.862798,0 days 02:26:44,0 days 02:02:26,1458.0,1


In [147]:
df_slice.query('time_diff > 62').sort_values(by='time_diff')

,bike_id,lon,lat,timestamp,prev_time,time_diff
18942,33e181328642bcbeea012e622944af1a,-77.050006,38.897873,0 days 02:29:46,0 days 02:27:45,121.0
13061,a7c567c6b92301b7d9d4c38d809e52a4,-76.974381,38.865621,0 days 01:59:23,0 days 01:57:22,121.0
11083,00d0b18c3dcda4d2a35d198617eb00bb,-76.998534,39.005582,0 days 01:49:16,0 days 01:47:15,121.0
6929,a2cb1ae754c1cb0b8b603fe19fd94a3b,-77.079031,38.839356,0 days 01:28:02,0 days 01:26:01,121.0
11554,96cd65315ee3fe78cfcccdda68b38f11,-77.041553,38.905730,0 days 01:52:18,0 days 01:50:17,121.0
...,...,...,...,...,...,...
5941,74d7ba8e97c6f64a9c69c92119b20f76,-77.108850,38.878888,0 days 01:22:59,0 days 01:05:48,1031.0
17936,88f1400c881f7c064bc08ea6500e180c,-76.999030,38.923270,0 days 02:24:42,0 days 02:07:29,1033.0
11773,06f9e7a5c0ea0f789f81949ed2171427,-77.093790,38.977720,0 days 01:53:19,0 days 01:35:06,1093.0
18617,edcce24b9bbe81fa4bb22e3d81ef6d19,-77.029277,38.905750,0 days 02:28:46,0 days 02:07:29,1277.0
